In [14]:
import pandas as pd
from scipy.io import arff
import numpy as np
import math
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
import os
import time
from tqdm.notebook import trange, tqdm

In [16]:
import warnings 
warnings.filterwarnings('ignore')

In [17]:
file_pwd = os.getcwd() + "\Data"
res = os.walk(file_pwd)
file_list = [i[2] for i in res][0]
file_list

['CM1.arff',
 'JM1.arff',
 'KC1.arff',
 'KC3.arff',
 'MC1.arff',
 'MC2.arff',
 'MW1.arff',
 'PC1.arff',
 'PC2.arff',
 'PC3.arff',
 'PC4.arff',
 'PC5.arff']

In [18]:
#输入data,输出log2D列的数据
def preprocess(df):
    head_list = df.columns.values.tolist()
    #标准化
    data_without_YN = df.drop("Defective",axis = 1)
    data_normalize = (data_without_YN-data_without_YN.mean())/(data_without_YN.std())
    data_normalize['Defective'] = df.Defective
    
    row_yes_data = df[df.Defective == b'Y']
    row_yes_data = row_yes_data.drop("Defective",axis = 1).values
    row_no_data =  df[df.Defective == b'N']
    row_no_data = row_no_data.drop("Defective",axis = 1).values
    
    yes_samples = data_normalize[data_normalize.Defective == b"Y"]
    yes_samples = yes_samples.drop("Defective",axis = 1)
    no_samples = data_normalize[data_normalize.Defective == b"N"]
    no_samples = no_samples.drop("Defective",axis = 1)
    k = len(no_samples)//len(yes_samples)
    
    yes_samples_array = yes_samples.values
    no_samples_array = no_samples.values
    array = [[np.sqrt(np.sum(np.square(x-y))) for y in no_samples_array]for x in yes_samples_array]
    array = np.array(array).argsort()[:,:k]
    w = {i:0 for i in range(yes_samples.shape[1])}

    for i in range(array.shape[0]):
        for j in array[i]:
            ds = np.abs(row_yes_data[i,:] - row_no_data[j,:])
            ds = pd.Series(ds).rank(method='min')
            for index in range(len(ds)):
                w[index] += ds[index]
    
    a = sorted(w.items(),key=lambda x:x[1],reverse=True)
    b = [i[0] for i in a ]
    c = np.array(head_list)
    column = list(c[b])
    df2 = df.loc[:,column].copy()
    
    d = df2.shape[1]
    log2d = math.ceil(math.log2(d))
    df2 = df2.iloc[:,:log2d]
    return df2

In [19]:
#返回十次十折交叉验证的平均auc
def SVM(data,label):
    clf = SVC(gamma='auto')
    auc_list = []
    data["label"] = label
    for i in tqdm(range(10)):
        data = data.sample(frac=1)
        scores = cross_val_score(clf,data.iloc[:,:-1],data.label,cv=10,scoring="roc_auc")
        auc_list.append(scores.mean())
    return np.mean(auc_list)

In [20]:
#贝叶斯分类
def NB(data,label):
    clf = MultinomialNB()
    auc_list = []
    data["label"] = label
    for i in tqdm(range(10)):
        data = data.sample(frac=1)
        scores = cross_val_score(clf,data.iloc[:,:-1],data.label,cv=10,scoring="roc_auc")
        auc_list.append(scores.mean())
    return np.mean(auc_list)

In [21]:
#决策树分类
def DT(data,label):
    clf = DecisionTreeClassifier()
    auc_list = []
    data["label"] = label
    for i in tqdm(range(10)):
        data = data.sample(frac=1)
        scores = cross_val_score(clf,data.iloc[:,:-1],data.label,cv=10,scoring="roc_auc")
        auc_list.append(scores.mean())
    return np.mean(auc_list)


In [36]:
real_start = time.clock()
for each in tqdm(file_list):
    res_list = []
    data = arff.loadarff('./data/{}'.format(each))
    df = pd.DataFrame(data[0])
    if df.columns[-1] == "label":
        df.rename(columns={'label':'Defective'},inplace=True) 
    defective = df.Defective.copy()
    defective[defective==b'N'] = 0
    defective[defective==b'Y'] = 1

    #得到排好序的数据
    data = preprocess(df)
    head_list = data.columns
    
    for every_feature in tqdm(head_list):
        start = time.clock()
        X = data.loc[:,head_list[0]:every_feature]
        label = defective.astype(int)
        svm_auc = SVM(X.copy(),label)
        destree_auc = DT(X.copy(),label)
        nb_auc = NB(X.copy(),label)
        print("*"*20)
        print("数据尺寸:{}".format(X.shape))
        print("文件名:{}".format("CM1"))
        print("feature:{}:{}".format(head_list[0],every_feature))
        print("SVM--->{}:".format(svm_auc))
        print("决策树--->{}:".format(destree_auc))
        print("贝叶斯--->{}".format(nb_auc))
        spend = (time.clock()-start)
        print("use time:{}".format(spend))
        print("="*20)
        make_dic = {
            "size":X.shape,
            "feature":every_feature,
            "SVM":svm_auc,
            "tree":destree_auc,
            "nb":nb_auc
        }
        res_list.append(make_dic)
    print(res_list)

    info = {key:[]for key in res_list[0].keys()}
    for one in res_list:
        for key,value in one.items():
            info[key].append(value)
    info = pd.DataFrame(info)
    info.to_csv("{}.csv".format(each))
print("总共耗时:",(time.clock()-real_start))


********************
数据尺寸:(344, 1)
文件名:CM1
feature:HALSTEAD_EFFORT:HALSTEAD_EFFORT
SVM--->0.5715940860215053:
决策树--->0.6030698924731184:
贝叶斯--->0.5
use time:4.678998700000193



********************
数据尺寸:(344, 2)
文件名:CM1
feature:HALSTEAD_EFFORT:HALSTEAD_PROG_TIME
SVM--->0.5818266129032258:
决策树--->0.6052204301075268:
贝叶斯--->0.3868225806451613
use time:4.443179099999725



********************
数据尺寸:(344, 3)
文件名:CM1
feature:HALSTEAD_EFFORT:HALSTEAD_VOLUME
SVM--->0.5454811827956989:
决策树--->0.55147311827957:
贝叶斯--->0.4737970430107527
use time:4.638903299999583



********************
数据尺寸:(344, 4)
文件名:CM1
feature:HALSTEAD_EFFORT:HALSTEAD_LENGTH
SVM--->0.5420094086021505:
决策树--->0.5230215053763441:
贝叶斯--->0.395497311827957
use time:4.65213279999989



********************
数据尺寸:(344, 5)
文件名:CM1
feature:HALSTEAD_EFFORT:NUM_OPERATORS
SVM--->0.5499153225806451:
决策树--->0.5103037634408603:
贝叶斯--->0.3794166666666667
use time:4.684788200000185


...


********************
数据尺寸:(9593, 12)
文件名:CM1
feature:HALSTEAD_EFFORT:BRANCH_COUNT
SVM--->0.6878812245949407:
决策树--->0.580905053137373:
贝叶斯--->0.3784825077188878
use time:870.8615461999998



********************
数据尺寸:(9593, 13)
文件名:CM1
feature:HALSTEAD_EFFORT:LOC_BLANK
SVM--->0.6879147396807033:
决策树--->0.588593629307457:
贝叶斯--->0.37798735552769963
use time:1158.8131246000012



********************
数据尺寸:(9593, 14)
文件名:CM1
feature:HALSTEAD_EFFORT:NUM_UNIQUE_OPERATORS
SVM--->0.6895774628190924:
决策树--->0.5894946711610889:
贝叶斯--->0.3794362659960797
use time:1035.5713605



********************
数据尺寸:(9593, 15)
文件名:CM1
feature:HALSTEAD_EFFORT:LOC_COMMENTS
SVM--->0.6900369149857565:
决策树--->0.5914623832985171:
贝叶斯--->0.37900125261529205
use time:943.8055820999998


KeyboardInterrupt: 

In [35]:
# real_start = time.clock()
# res_list = []
# for each in file_list:
#     data = arff.loadarff('./data/{}'.format(each))
#     df = pd.DataFrame(data[0])
#     if df.columns[-1] == "label":
#         df.rename(columns={'label':'Defective'},inplace=True) 
#     defective = df.Defective.copy()
#     defective[defective==b'N'] = 0
#     defective[defective==b'Y'] = 1
    
#     start = time.clock()
#     #分为数据和标签
#     data = preprocess(df)
#     label = defective.astype(int)
#     svm_auc = SVM(data,label)
#     destree_auc = DT(data,label)
#     nb_auc = NB(data,label)
#     print("*"*20)
#     print("数据尺寸:{}".format(data.shape))
#     print("文件名:{}".format(each))
#     print("log2D:{}".format(data.shape[1]))
#     print("SVM--->{}:".format(svm_auc))
#     print("决策树--->{}:".format(destree_auc))
#     print("贝叶斯--->{}".format(nb_auc))
#     spend = (time.clock()-start)
#     print("use time:{}".format(spend))
#     print("="*20)
#     make_dic = {
#         "size":data.shape,
#         "name":each,
#         "log2D":data.shape[1],
#         "SVM":svm_auc,
#         "tree":destree_auc,
#         "nb":nb_auc
#     }
#     res_list.append(make_dic)
# print("总共耗时:",(time.clock()-real_start))
# print(res_list)


In [11]:
# data = {key:[]for key in res_list[0].keys()}
# for one in res_list:
#     for key,value in one.items():
#         data[key].append(value)
# data.pop("size")
# data = pd.DataFrame(data,index=range(1,13))
# data.to_csv("log2D.csv")